In [1]:
using Laplacians

objc[25419]: Class TKApplication is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[25419]: Class TKMenu is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[25419]: Class TKContentView is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.
objc[25419]: Class TKWindow is implemented in both /System/Library/Frameworks/Tk.framework/Versions/8.5/Tk and /Users/serbanstan/anaconda/lib/libtk8.5.dylib. One of the two will be used. Which one is undefined.


In [12]:
include("treeSolver.jl")
include("IO.jl")
include("generators.jl")
include("toHighPrecision.jl")
include("verbosePCG.jl")

pcgWorker (generic function with 1 method)

In [13]:
pwd()

"/Users/serbanstan/git/TreePCG/julia"

In [14]:
graphp = "../graphs/se_7/"

"../graphs/se_7/"

In [15]:
a = readFromFile(graphp * "graph.mtx");

In [16]:
tree = readFromFile(graphp * "tree.mtx");

In [17]:
x = readFromFile(graphp * "x.vec");

In [18]:
la = lap(a);
x = x - mean(x)
b = la * x; b = b - mean(b);

In [19]:
@time F = treeSolver(tree)
@time F(b);

  0.024466 seconds (7.88 k allocations: 3.247 MB, 18.47% gc time)
  0.033744 seconds (172.03 k allocations: 3.666 MB)


In [ ]:
# a run on standard Float64 data types
numIts = 10000;
@time myx64,dbg = pcgV(la, b, F, x, maxits=numIts, verbose=true);
norm(lap(a) * myx64 - b) / norm(b)
insert!(dbg, 1, "$(numIts) iterations")
insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
writeToFile(graphp * "_log_julia_default.txt", dbg)

Working on iteration 10 with a-norm error 1.2498752286060086e7
Working on iteration 20 with a-norm error 6.470011286297602e6
Working on iteration 30 with a-norm error 3.9502254789566426e6
Working on iteration 40 with a-norm error 2.776332091328997e6
Working on iteration 50 with a-norm error 2.2040920242817006e6
Working on iteration 60 with a-norm error 1.8082354640398778e6
Working on iteration 70 with a-norm error 1.5107124119369911e6
Working on iteration 80 with a-norm error 1.307546396891764e6
Working on iteration 90 with a-norm error 1.1596789184401927e6
Working on iteration 100 with a-norm error 1.0435765953976188e6
Working on iteration 110 with a-norm error 929553.7184270624
Working on iteration 120 with a-norm error 836240.7535253817
Working on iteration 130 with a-norm error 786210.4993211476
Working on iteration 140 with a-norm error 724151.3939733849
Working on iteration 150 with a-norm error 668929.5720359728
Working on iteration 160 with a-norm error 618895.3468936416
Workin

In [ ]:
numIts = 10000
for hp in [64,256,4096]
    set_bigfloat_precision(hp)
    laHP = toHighPrecision(la; precision=hp)
    treeHP = toHighPrecision(tree; precision=hp)
    xHP = toHighPrecision(x; precision=hp);
    bHP = toHighPrecision(b; precision=hp);
    FHP = treeSolver(treeHP)
    
    @time myxHP,dbg = pcgV(laHP, bHP, FHP, xHP, maxits=numIts, verbose=true);
    insert!(dbg, 1, "$(numIts) iterations")
    insert!(dbg, 1, "anorm = sqrt((x-truex)' * la * (x-truex)), err2_a = norm(la*x-b) / norm(b), err2_b = norm(x-truex)")
    writeToFile(graphp * "_log_julia_$(hp).txt", dbg)
end

Working on iteration 10 with a-norm error 1.452370308300438e9
Working on iteration 20 with a-norm error 7.219248657308702e8
Working on iteration 30 with a-norm error 4.391977195583564e8
Working on iteration 40 with a-norm error 3.099648890811332e8
Working on iteration 50 with a-norm error 2.4318406112023753e8
Working on iteration 60 with a-norm error 1.8706593154843011e8
Working on iteration 70 with a-norm error 1.6108499847329772e8
Working on iteration 80 with a-norm error 1.3942649000701252e8
Working on iteration 90 with a-norm error 1.231320684004477e8
Working on iteration 100 with a-norm error 1.0943336527149726e8
Working on iteration 110 with a-norm error 9.973292625479415e7
Working on iteration 120 with a-norm error 9.103542204147653e7
Working on iteration 130 with a-norm error 8.221586903093137e7
Working on iteration 140 with a-norm error 7.618358641998267e7
Working on iteration 150 with a-norm error 7.095497446906851e7
Working on iteration 160 with a-norm error 6.58857500050936